<a href="https://colab.research.google.com/github/lukmandev/NBC-Twitter/blob/master/Naive_Bayes_Text_Classification_Tweet_Prostitute_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pada notebook ini menampilkan metode klasifikasi dokumen teks menggunakan metode Naive Bayes untuk melakukan prediksi data twitter yang termasuk dalam kategori prostitusi atau bukan prostitusi.

Dataset diambil dari twitter sebanyak 40000 data termasuk dalam kategori true (prostitusi) sebanyak 20000 dan false (bukan prostitusi) sebanyak 20000 data.


Pertama, import modul python yang digunakan untuk proses preprocessing dan pembuatan model untuk dilakukan klasifikasi.

In [1]:
import pandas as pd
import string
import numpy as np
import re
import random
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.naive_bayes import MultinomialNB

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

Import semua dataset yang akan dilakukan klasifikasi


In [2]:
data_train_true = pd.read_excel('twitter-prostitute.xlsx')
data_train_false = pd.read_excel('twitter-not-prostitute.xlsx')

Gabungkan data true (prostitute) dan data false (not prostitute)

In [3]:
dataset = pd.concat([data_train_true, data_train_false], ignore_index = True)

In [4]:
print(dataset[:5])

                    id                 date       username  \
0  1255052854739398658  2020-04-28 00:00:00   rina11091996   
1  1255052799798202373  2020-04-28 00:00:00   rina11091996   
2  1255052613646573569  2020-04-28 00:00:00  viollasyantik   
3  1255052558667661312  2020-04-28 00:00:00    lyannyhijab   
4  1255052557061287938  2020-04-28 00:00:00      dheajogja   

                                               tweet  status  
0  AvaiL BO yaa beb😙\nWA 0831 9315 9762\n#AvailJo...       1  
1  Include exclude Ready beb \nWa 0831 9315 9762\...       1  
2  AvaiL Jogja Minat DM aja😍\nFasht Respon.\n#Ava...       1  
3  MAEN SANTAI GA BURU" \nFULL SERVICE NO ANAL US...       1  
4  New bie...Ready ya..2 slot aja 085647266101\n#...       1  


In [5]:
print(dataset['tweet'][:5])

0    AvaiL BO yaa beb😙\nWA 0831 9315 9762\n#AvailJo...
1    Include exclude Ready beb \nWa 0831 9315 9762\...
2    AvaiL Jogja Minat DM aja😍\nFasht Respon.\n#Ava...
3    MAEN SANTAI GA BURU" \nFULL SERVICE NO ANAL US...
4    New bie...Ready ya..2 slot aja 085647266101\n#...
Name: tweet, dtype: object


In [6]:
print(len(dataset))

40000


Setelah data berhasil dibuka, saatnya melakukan preprocessing pada teks. Yang akan dilakukan preprocessing yaitu data pada kolom **tweet** dimana data ini berisi tweet yang diambil dari twitter.

Ada beberapa process preprocessing yaitu cleaning text atau membersihkan text dari noise dan tokenizing yaitu memecah semua text menjadi per kata.

In [7]:
stopwords_file = open("stopwords-id.txt", 'r')
stopwords = [x.strip() for x in stopwords_file.readlines()]
stopwords.extend(['by', 'rt', 'via'])

def cleaning(text):
	text = re.sub(r'<[^>]+>', '', text) #delete html tags
	text = re.sub(r'\S*twitter.com\S*', '', text)   #delete twitter image
	text = re.sub(r'https?://[A-Za-z0-9./]+','',text) #delete url
	text = re.sub(r'@[A-Za-z0-9]+','',text) #delete user mention
	text = re.sub(r'#[A-Za-z0-9]+','',text) #delete twitter hashtag
	text = re.sub(r'(?:(?:\d+,?)+(?:\.?\d+)?)','', text) #delete number
	text = re.sub(r"[^a-zA-Z]", " ", text) #only accept alphabet char
	text = re.sub(r"(\w)(\1{2,})", r'\1', text) #delete repeated char
	text = re.sub(r"\b[a-zA-Z]\b", "", text) #remove single character
	text = text.lower() #change to lowercase
	text = stemmer.stem(text) #stemming
	return text

def tokenize(text):
	#disini diisi dengan stop words
	words = text.split();
	words = [w for w in words if w not in stopwords]
	return words

Cleaning text pada data tweet

In [8]:
dataset['tweet'] = dataset.tweet.map(lambda x: cleaning(x))

In [9]:
print(dataset['tweet'][:5])

0                                  avail bo yaa beb wa
1                         include exclude ready beb wa
2                avail jogja minat dm aja fasht respon
3    maen santai ga buru full service no anal use caps
4                            new bie ready ya slot aja
Name: tweet, dtype: object


Tokenizing text pada data tweet

In [10]:
dataset['tweet'] = dataset.tweet.apply(lambda x: tokenize(x))

In [11]:
print(dataset['tweet'][:5])

0                            [avail, bo, yaa, beb, wa]
1                   [include, exclude, ready, beb, wa]
2        [avail, jogja, minat, dm, aja, fasht, respon]
3    [maen, santai, ga, buru, full, service, no, an...
4                     [new, bie, ready, ya, slot, aja]
Name: tweet, dtype: object


Menggabungkan semua tweet menggunakan spasi untuk dilakukan tahap selanjutnya yaitu pembuatan vektor

In [12]:
dataset['tweet'] = dataset.tweet.apply(lambda x: ' '.join(x))

Memasukkan label ke array

In [13]:
dataset['label'] = dataset.status.map(lambda x: x)

In [14]:
count_vect = CountVectorizer()
counts = count_vect.fit_transform(dataset['tweet'])

In [15]:
print(counts[:5])

  (0, 1580)	1
  (0, 3027)	1
  (0, 27875)	1
  (0, 2240)	1
  (0, 27195)	1
  (1, 2240)	1
  (1, 27195)	1
  (1, 10127)	1
  (1, 7044)	1
  (1, 20865)	1
  (2, 1580)	1
  (2, 11145)	1
  (2, 15754)	1
  (2, 6087)	1
  (2, 444)	1
  (2, 7244)	1
  (2, 21200)	1
  (3, 14462)	1
  (3, 22030)	1
  (3, 7788)	1
  (3, 3578)	1
  (3, 7738)	1
  (3, 22835)	1
  (3, 17619)	1
  (3, 879)	1
  (3, 26835)	1
  (3, 3787)	1
  (4, 20865)	1
  (4, 444)	1
  (4, 16945)	1
  (4, 2742)	1
  (4, 27874)	1
  (4, 23503)	1


Disini mentransformasikan semua feature kata menggunakan TF IDF

In [16]:
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

In [17]:
print(counts[:5])

  (0, 27875)	0.5911705953129078
  (0, 27195)	0.2822194474686629
  (0, 3027)	0.3843499205847922
  (0, 2240)	0.47674485769290337
  (0, 1580)	0.44255958911507276
  (1, 27195)	0.2866682929106074
  (1, 20865)	0.38428108764986996
  (1, 10127)	0.49885978763514066
  (1, 7044)	0.5355187636154758
  (1, 2240)	0.4842601590165411
  (2, 21200)	0.3753169580936991
  (2, 15754)	0.27032017850061374
  (2, 11145)	0.2156594729911206
  (2, 7244)	0.7575121245337826
  (2, 6087)	0.18561664533035457
  (2, 1580)	0.24615315912467356
  (2, 444)	0.2658677080531679
  (3, 26835)	0.3987895175091566
  (3, 22835)	0.3340995998990038
  (3, 22030)	0.2971735280329197
  (3, 17619)	0.18397171528605755
  (3, 14462)	0.38405376089842264
  (3, 7788)	0.20809474894554508
  (3, 7738)	0.2971735280329197
  (3, 3787)	0.3588993202178047
  (3, 3578)	0.2611730028167115
  (3, 879)	0.36201353464718405
  (4, 27874)	0.21768241478277015
  (4, 23503)	0.2879370816285136
  (4, 20865)	0.23767142901177377
  (4, 16945)	0.4494329267607444
  (4, 2742)

Kemudian setelah semua kata memiliki bobot tersendiri sesuai dengan transformasi vektor TF IDF, disini pembagian data training dan data testing dengan alokasi 80% train:20% test

In [18]:
feature_train, feature_test, target_train, target_test = train_test_split(counts, dataset['label'], train_size=0.8, test_size=0.2, random_state=52)

In [19]:
model = MultinomialNB()
model.fit(feature_train, target_train)
predicted = model.predict(feature_test)

Untuk menghitung akurasi model yang sudah dibuat, dapat menggunakan accuracy_score dari scikitlearn

In [20]:
accuracy = accuracy_score(target_test, predicted)

In [21]:
print(f'Naive Bayes Accuracy : {accuracy*100}%')

Naive Bayes Accuracy : 98.45%


Untuk pengujian, menggunakan confusion matrix sebagai berikut ini:

In [22]:
c_matrix = pd.DataFrame(
    confusion_matrix(target_test, predicted, labels=[1, 0]),
    index = ['Actual:True', 'Actual:False'],
    columns = ['Pred:True', 'Pred:False']
);

In [23]:
print(c_matrix)

              Pred:True  Pred:False
Actual:True        3928          59
Actual:False         65        3948


Untuk melihat laporan klasifikasi, menggunakan classification_report 

In [24]:
c_report = classification_report(target_test, predicted)

In [25]:
print(c_report)

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      4013
           1       0.98      0.99      0.98      3987

    accuracy                           0.98      8000
   macro avg       0.98      0.98      0.98      8000
weighted avg       0.98      0.98      0.98      8000



Kemudian kita lakukan prediksi data untuk data baru

In [26]:
def pred_text(score):
  if score == 0:
    return 'False (Not Prostitute)'
  else:
    return 'True (Prostitute)'

input_text = 'Semoga covid-19 ini segera berakhir dan semua kegiatan dapat kembali seperti sedia kala'
input_text = tokenize(cleaning(input_text))
new_counts = count_vect.transform(input_text)
pred = model.predict(new_counts)
print(pred_text(pred[0]))


False (Not Prostitute)


In [27]:
input_text = 'area #Jogja cod no dp free cancel include room'
input_text = tokenize(cleaning(input_text))
new_counts = count_vect.transform(input_text)
pred = model.predict(new_counts)
print(pred_text(pred[0]))

True (Prostitute)
